In [1]:
from API_call import API_call
from gamePks import *
import statsapi as mlb
import pandas as pd

from tables import *
import relationships

from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=db.db_engine)
session = Session()

Engine(sqlite://///Users/schlinkertc/code/MLB/mlb.db)


In [2]:
engine = db.db_engine
stmt="""
select
    *
from 
    matchups

"""

In [3]:
matchups = pd.read_sql(stmt,engine)

In [4]:
len(matchups['gamePk'].unique())

3953

In [5]:
stmt="""
SELECT 
    count(detailedState) as count,
    pk,
    detailedState
FROM 
    games
where 
    type = 'R'
    and 
    season = 2019
group by 
    detailedState
"""

In [6]:
df = pd.read_sql(stmt,engine)
df

,count,pk,detailedState
0,1,567304,Cancelled
1,10,567462,Completed Early: Rain
2,2419,566083,Final


In [7]:
pk = 567491
db.insert_game(pk,replace=True)

In [198]:
# query required offensive stats for Rickey's equation 
stmt=f"""
SELECT
    players.fullName,
    players.id,
    p.gamePk,
    p.home_away,
    sum(s.atBats) as atBats,
    sum(s.baseOnBalls)+sum(s.intentionalWalks) as BB,
    sum(s.hitByPitch) as HBP,
    sum(s.hits) as hits,
    sum(s.totalBases) as tb,
    sum(s.runs) as runs
FROM 
    game_player_links p
INNER JOIN 
    game_batting_stats s ON p.person_id=s.person_id
INNER JOIN 
    players on players.id=p.person_id
WHERE
    p.gamePk = {pk} AND players.primaryPosition_name != 'Pitcher'
AND 
    s.gamePk
IN
    (Select pk from games where season = 2019 and type = 'R')
group by 
    players.fullName
    """

In [199]:
engine = db.db_engine

In [200]:
offense=pd.read_sql(stmt,engine)
offense

,fullName,id,gamePk,home_away,atBats,BB,HBP,hits,tb,runs
0,Aaron Hicks,543305,567491,home,221.0,31.0,0.0,52.0,98.0,41.0
1,Adeiny Hechavarria,588751,567491,away,203.0,14.0,3.0,49.0,90.0,34.0
2,Amed Rosario,642708,567491,away,616.0,33.0,3.0,177.0,266.0,75.0
3,Austin Romine,519222,567491,home,228.0,10.0,0.0,64.0,100.0,29.0
4,Brett Gardner,458731,567491,home,491.0,52.0,4.0,123.0,247.0,86.0
5,Cameron Maybin,457727,567491,home,239.0,30.0,0.0,68.0,118.0,48.0
6,Carlos Gomez,460576,567491,away,86.0,7.0,3.0,17.0,29.0,10.0
7,Clint Frazier,640449,567491,home,225.0,17.0,2.0,60.0,110.0,31.0
8,DJ LeMahieu,518934,567491,home,602.0,46.0,2.0,197.0,312.0,109.0
9,Didi Gregorius,544369,567491,home,324.0,18.0,1.0,77.0,143.0,47.0


In [189]:
# query required defensive stats for Rickey's equation 
stmt=f"""
SELECT 
    p.home_away,
    players.fullName,
    sum(s.hits) as hits,
    sum(s.atBats) as atBats,
    sum(s.baseOnBalls) as walks,
    sum(s.hitBatsmen) as HBP,
    sum(s.earnedRuns) as er,
    sum(s.inningsPitched)/sum(s.gamesPitched) > 3 
        and
        sum(s.gamesStarted)/sum(s.gamesPitched) > .9
        as starter
FROM 
    game_player_links p
INNER JOIN 
    game_pitching_stats s ON p.person_id=s.person_id
INNER JOIN 
    players on players.id=p.person_id
INNER JOIN 
    games g ON g.pk=s.gamePk
WHERE
    p.gamePk = {pk} AND players.primaryPosition_name = 'Pitcher'
AND 
    g.season = 2019 and g.type = 'R'
group by 
    players.fullName
    """

In [190]:
defense = pd.read_sql(stmt,engine)
defense

,home_away,fullName,hits,atBats,walks,HBP,er,starter
0,home,Adam Ottavino,47.0,237.0,40.0,2.0,14.0,0
1,home,Aroldis Chapman,38.0,205.0,25.0,1.0,14.0,0
2,home,CC Sabathia,112.0,422.0,39.0,3.0,59.0,1
3,home,Chad Green,66.0,267.0,19.0,6.0,32.0,0
4,home,Chance Adams,39.0,111.0,11.0,2.0,24.0,0
5,home,David Hale,39.0,148.0,7.0,1.0,13.0,0
6,away,Drew Gagnon,34.0,108.0,7.0,2.0,22.0,0
7,away,Edwin Diaz,58.0,225.0,22.0,4.0,36.0,0
8,away,Hector Santiago,42.0,140.0,22.0,0.0,25.0,0
9,home,J.A. Happ,160.0,621.0,49.0,5.0,88.0,1


once more but in aggregate by teams

In [313]:
#offense
stmt=f"""
SELECT
    p.gamePk,
    p.home_away,
    (
        sum(s.hits) +
        sum(s.baseOnBalls) +
        sum(s.intentionalWalks) +
        sum(s.hitByPitch) 
    ) /
    (   sum(s.atBats) + 
        sum(s.baseOnBalls) +
        sum(s.intentionalWalks) +
        sum(s.hitByPitch)
    ) as offense,
    
    (sum(s.totalBases) - sum(s.hits)) / sum(s.atBats) as power,
    sum(s.runs) / 
    (
        sum(s.hits) +
        sum(s.baseOnBalls) +
        sum(s.intentionalWalks) +
        sum(s.hitByPitch) 
    ) as clutch
FROM 
    game_player_links p
INNER JOIN 
    game_batting_stats s ON p.person_id=s.person_id
INNER JOIN 
    players on players.id=p.person_id
WHERE
    p.gamePk = {pk} AND players.primaryPosition_name != 'Pitcher'
AND 
    s.gamePk
IN
    (Select pk from games where season = 2019 and type = 'R' and dateTime < "2019-06-11")
group by 
    p.home_away
    """

In [314]:
off = pd.read_sql(stmt,engine)
off.set_index('home_away',inplace=True)
off

,gamePk,offense,power,clutch
home_away,,,,
away,567491,0.344774,0.192173,0.366516
home,567491,0.339329,0.192725,0.405444


In [315]:
# for relievers
stmt=f"""
SELECT 
    p.home_away,
    sum(s.hits) / sum(s.atBats) as r_oppBa,
    ( sum(s.baseOnBalls) + sum(s.hitBatsmen) )
    /
    ( sum(s.atBats) +  sum(s.baseOnBalls) + sum(s.hitBatsmen) ) as r_oppBb,
    sum(s.earnedRuns) 
    /
    ( sum(s.hits) +  sum(s.baseOnBalls) + sum(s.hitBatsmen) ) as r_oppEr

FROM 
    game_player_links p
INNER JOIN 
    game_pitching_stats s ON p.person_id=s.person_id
INNER JOIN 
    players on players.id=p.person_id
INNER JOIN 
    games g ON g.pk=s.gamePk
WHERE
    p.gamePk = {pk} AND players.primaryPosition_name = 'Pitcher'
    AND 
    g.season = 2019 and g.type = 'R' and g.dateTime < "2019-06-11"
group by 
    p.home_away
having
    sum(s.inningsPitched)/sum(s.gamesPitched) < 3 
    AND
    sum(s.gamesStarted)/sum(s.gamesPitched) < .9
    """

In [316]:
def_relief = pd.read_sql(stmt,engine)
def_relief.set_index('home_away',inplace=True)
def_relief

,r_oppBa,r_oppBb,r_oppEr
home_away,,,
away,0.250000,0.087844,0.353933
home,0.234818,0.090000,0.336222


In [302]:
# for starter
stmt=f"""
SELECT 
    p.home_away,
    sum(s.hits) / sum(s.atBats) as s_oppBa,
    ( sum(s.baseOnBalls) + sum(s.hitBatsmen) )
    /
    ( sum(s.atBats) +  sum(s.baseOnBalls) + sum(s.hitBatsmen) ) as s_oppBb,
    sum(s.earnedRuns) 
    /
    ( sum(s.hits) +  sum(s.baseOnBalls) + sum(s.hitBatsmen) ) as s_oppEr

FROM 
    game_player_links p
INNER JOIN 
    game_pitching_stats s ON p.person_id=s.person_id
INNER JOIN 
    players on players.id=p.person_id
INNER JOIN 
    games g ON g.pk=s.gamePk
WHERE
    p.gamePk = {pk}
    AND 
    g.season = 2019 and g.type = 'R' and g.dateTime < "2019-06-11"
    AND
    players.id = g.home_probablePitcher or players.id = away_probablePitcher
group by 
    p.home_away
    """

In [303]:
starters = pd.read_sql(stmt,engine)
starters.set_index('home_away',inplace=True)
starters

,s_oppBa,s_oppBb,s_oppEr
home_away,,,
away,0.258778,0.089064,0.368402
home,0.258918,0.089008,0.368209


In [304]:
df = pd.concat([off,def_relief,starters],sort=False,axis=1)

In [305]:
df

,gamePk,offense,power,clutch,r_oppBa,r_oppBb,r_oppEr,s_oppBa,s_oppBb,s_oppEr
home_away,,,,,,,,,,
away,567491,0.344774,0.192173,0.366516,0.250000,0.087844,0.353933,0.258778,0.089064,0.368402
home,567491,0.339329,0.192725,0.405444,0.234818,0.090000,0.336222,0.258918,0.089008,0.368209


In [306]:
records = df.to_dict(orient='index')

In [307]:
d = {}
for team in ['home','away']:
    keys = records[team].keys()
    for k in keys:
        if k != 'gamePk':
            d[f"{team}_{k}"] = records[team][k]
        d['gamePk']=records[team]['gamePk']

In [308]:
d

{'gamePk': 567491,
 'home_offense': 0.33932912007778315,
 'home_power': 0.19272529858849077,
 'home_clutch': 0.4054441260744986,
 'home_r_oppBa': 0.23481781376518218,
 'home_r_oppBb': 0.09,
 'home_r_oppEr': 0.3362218370883882,
 'home_s_oppBa': 0.2589178316117167,
 'home_s_oppBb': 0.08900760149141955,
 'home_s_oppEr': 0.36820866398613367,
 'away_offense': 0.34477379095163807,
 'away_power': 0.19217289719626168,
 'away_clutch': 0.3665158371040724,
 'away_r_oppBa': 0.25,
 'away_r_oppBb': 0.0878438331854481,
 'away_r_oppEr': 0.3539325842696629,
 'away_s_oppBa': 0.2587778647469107,
 'away_s_oppBb': 0.08906400106108556,
 'away_s_oppEr': 0.3684018896397273}

In [291]:
game_record = session.query(Game).filter(Game.pk == pk).one()

In [293]:
dt.strftime(game_record.dateTime,"%Y-%m-%d")

'2019-06-11'

In [309]:
game_record.pk

567491

In [327]:
def get_game_features(game_record):
    date = dt.strftime(game_record.dateTime,"%Y-%m-%d")
    pk = game_record.pk
    
    #offense
    stmt=f"""
    SELECT
        p.gamePk,
        p.home_away,
        (
            sum(s.hits) +
            sum(s.baseOnBalls) +
            sum(s.intentionalWalks) +
            sum(s.hitByPitch) 
        ) /
        (   sum(s.atBats) + 
            sum(s.baseOnBalls) +
            sum(s.intentionalWalks) +
            sum(s.hitByPitch)
        ) as offense,

        (sum(s.totalBases) - sum(s.hits)) / sum(s.atBats) as power,
        sum(s.runs) / 
        (
            sum(s.hits) +
            sum(s.baseOnBalls) +
            sum(s.intentionalWalks) +
            sum(s.hitByPitch) 
        ) as clutch
    FROM 
        game_player_links p
    INNER JOIN 
        game_batting_stats s ON p.person_id=s.person_id
    INNER JOIN 
        players on players.id=p.person_id
    WHERE
        p.gamePk = {pk} AND players.primaryPosition_name != 'Pitcher'
    AND 
        s.gamePk
    IN
        (Select pk from games where season = 2019 and type = 'R' and dateTime < "{date}")
    group by 
        p.home_away
        """
    off = pd.read_sql(stmt,engine)
    off.set_index('home_away',inplace=True)
    
    # for relievers
    stmt=f"""
    SELECT 
        p.home_away,
        sum(s.hits) / sum(s.atBats) as r_oppBa,
        ( sum(s.baseOnBalls) + sum(s.hitBatsmen) )
        /
        ( sum(s.atBats) +  sum(s.baseOnBalls) + sum(s.hitBatsmen) ) as r_oppBb,
        sum(s.earnedRuns) 
        /
        ( sum(s.hits) +  sum(s.baseOnBalls) + sum(s.hitBatsmen) ) as r_oppEr

    FROM 
        game_player_links p
    INNER JOIN 
        game_pitching_stats s ON p.person_id=s.person_id
    INNER JOIN 
        players on players.id=p.person_id
    INNER JOIN 
        games g ON g.pk=s.gamePk
    WHERE
        p.gamePk = {pk} AND players.primaryPosition_name = 'Pitcher'
        AND 
        g.season = 2019 and g.type = 'R' and g.dateTime < "{date}"
    group by 
        p.home_away
    having
        sum(s.inningsPitched)/sum(s.gamesPitched) < 3 
        AND
        sum(s.gamesStarted)/sum(s.gamesPitched) < .9
        """
    relief = pd.read_sql(stmt,engine)
    relief.set_index('home_away',inplace=True)
    
    # for starter
    stmt=f"""
    SELECT 
        p.home_away,
        sum(s.hits) / sum(s.atBats) as s_oppBa,
        ( sum(s.baseOnBalls) + sum(s.hitBatsmen) )
        /
        ( sum(s.atBats) +  sum(s.baseOnBalls) + sum(s.hitBatsmen) ) as s_oppBb,
        sum(s.earnedRuns) 
        /
        ( sum(s.hits) +  sum(s.baseOnBalls) + sum(s.hitBatsmen) ) as s_oppEr

    FROM 
        game_player_links p
    INNER JOIN 
        game_pitching_stats s ON p.person_id=s.person_id
    INNER JOIN 
        players on players.id=p.person_id
    INNER JOIN 
        games g ON g.pk=s.gamePk
    WHERE
        p.gamePk = {pk}
        AND 
        g.season = 2019 and g.type = 'R' and g.dateTime < "{date}"
        AND
        players.id = g.home_probablePitcher or players.id = away_probablePitcher
    group by 
        p.home_away
        """
    starters = pd.read_sql(stmt,engine)
    starters.set_index('home_away',inplace=True)
    
    df = pd.concat([off,relief,starters],sort=False,axis=1)
    records = df.to_dict(orient='index')
    
    d = {}
    for team in ['home','away']:
        keys = records[team].keys()
        for k in keys:
            if k != 'gamePk':
                d[f"{team}_{k}"] = records[team][k]
            d['gamePk']=records[team]['gamePk']
    return d

In [328]:
get_game_features(game_record)

{'gamePk': 567491,
 'home_offense': 0.33932912007778315,
 'home_power': 0.19272529858849077,
 'home_clutch': 0.4054441260744986,
 'home_r_oppBa': 0.23481781376518218,
 'home_r_oppBb': 0.09,
 'home_r_oppEr': 0.3362218370883882,
 'home_s_oppBa': 0.2589178316117167,
 'home_s_oppBb': 0.08900760149141955,
 'home_s_oppEr': 0.36820866398613367,
 'away_offense': 0.34477379095163807,
 'away_power': 0.19217289719626168,
 'away_clutch': 0.3665158371040724,
 'away_r_oppBa': 0.25,
 'away_r_oppBb': 0.0878438331854481,
 'away_r_oppEr': 0.3539325842696629,
 'away_s_oppBa': 0.2587778647469107,
 'away_s_oppBb': 0.08906400106108556,
 'away_s_oppEr': 0.3684018896397273}

In [340]:
april_1 = dt(2019,4,1)

In [342]:
games = session.query(Game).filter(Game.dateTime>april_1).all()

In [344]:
data = [get_game_features(game) for game in games]

In [347]:
data = pd.DataFrame.from_records(data)

In [348]:
data.to_csv('dataset.csv',index=False)

In [350]:
162*15

2430

In [353]:
data.describe()

,gamePk,home_offense,home_power,home_clutch,home_r_oppBa,home_r_oppBb,home_r_oppEr,home_s_oppBa,home_s_oppBb,home_s_oppEr,away_offense,away_power,away_clutch,away_r_oppBa,away_r_oppBb,away_r_oppEr,away_s_oppBa,away_s_oppBb,away_s_oppEr
count,2414.000000,2414.000000,2414.000000,2414.000000,2414.000000,2414.000000,2414.000000,2414.000000,2414.000000,2414.000000,2414.000000,2414.000000,2414.000000,2410.000000,2410.000000,2410.000000,2414.000000,2414.000000,2414.000000
mean,566679.165286,0.335430,0.186850,0.382265,0.243750,0.095245,0.345964,0.258919,0.089008,0.368215,0.335239,0.187139,0.382403,0.243688,0.095044,0.345333,0.258782,0.089062,0.368407
std,4162.442675,0.022365,0.029101,0.036053,0.017985,0.013185,0.033478,0.000005,0.000003,0.000009,0.021791,0.028983,0.036727,0.018156,0.013245,0.032884,0.000005,0.000003,0.000009
min,564734.000000,0.220000,0.055556,0.181818,0.155556,0.055556,0.147059,0.258899,0.088992,0.368181,0.245283,0.067073,0.145833,0.133333,0.031847,0.191176,0.258763,0.089046,0.368374
25%,565451.250000,0.321733,0.170375,0.368592,0.233316,0.086941,0.326743,0.258917,0.089006,0.368211,0.322136,0.171162,0.369235,0.232663,0.086743,0.328250,0.258780,0.089061,0.368402
50%,566189.500000,0.337705,0.189334,0.383164,0.245027,0.095435,0.345703,0.258920,0.089008,0.368215,0.336945,0.189253,0.383923,0.244696,0.095177,0.345895,0.258783,0.089063,0.368407
75%,566928.750000,0.350127,0.205445,0.401802,0.255459,0.102969,0.365309,0.258922,0.089010,0.368220,0.349450,0.204722,0.401605,0.255349,0.102467,0.363478,0.258785,0.089064,0.368412
max,599377.000000,0.455056,0.324138,0.510870,0.345455,0.198198,0.534884,0.258938,0.089015,0.368253,0.436090,0.349398,0.524324,0.308271,0.192000,0.472973,0.258801,0.089069,0.368445
